# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откроем файл с данными и изучим общую информацию.  <a class="anchor" id="first-bullet"></a>

In [995]:
import pandas as pd
df = pd.read_csv('/datasets/data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [996]:
df.head()


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Рассмотрим полученную информацию подробнее.

Всего в таблице 12 столбцов, типы данных: float64(2), int64(5), object(5)

Подробно разберём, какие в df столбцы и какую информацию они содержат:

- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита

Количество значений в столбцах различается. Это говорит о том, что в данных есть Null значения.
Столбец education содержит дубликаты с различным регистром.
Столбец days_employed содержит отрицательные значения.

Для получения основных статистик по выборке используем метод describe:

In [997]:
df.describe(percentiles=None, include=None, exclude=None)

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


 <b>Вывод:</b>

Каждая строка таблицы содержит информацию о клиенте, необходимо избавиться от пропущенных значений, привести столбец education к общему регистру, проверить данные в столбце days_employed, а также проверить данные на аномалии: в столбце days_employed присутствуют выбросы, имеются отрицательные значения в столбцах children и days_employed.
    
Наиболее важными для исследования являются столбцы debt, children и family_status

## Шаг 2. Предобработка данных <a class="anchor" id="second-bullet"></a>

### Обработка пропусков

Проверим общее количество пропусков:

In [998]:
df.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Для некоторых клиентов представлена не вся информация. У 2174 клиентов пропущен общий трудовой стаж в днях и ежемесячный доход.
Так как количество строк с пропущенными значениями составляет 10% от всех данных, заменим пропущенные NaN значения на "0" без удаления строк. 

In [999]:
df = df.fillna(0)

Проверим общую информацию:

In [1000]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


#### Проверка на аномалии

Проанализиируем аномально высокие значения стажа:

In [1001]:
df['days_employed'].describe()

count     21525.000000
mean      56678.874622
std      134870.763085
min      -18388.949901
25%       -2518.168900
50%        -982.531720
75%           0.000000
max      401755.400475
Name: days_employed, dtype: float64

Эмпирически найдено число дней 320000 ниже которого данные похожи на реальные.

In [1002]:
df[(df['days_employed'] > 0) & (df['days_employed'] < 320000)].sort_values(by = 'days_employed')

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


In [1003]:
df[(df['days_employed'] > 0) & (df['days_employed'] < 320000)]['income_type'].value_counts()

Series([], Name: income_type, dtype: int64)

In [1004]:
df[df['days_employed'] > 320000]['income_type'].value_counts()

пенсионер      3443
безработный       2
Name: income_type, dtype: int64

Стаж со значением превышающим 320000 дней в основном соответствует клиентам из категории "пенсионер".
Необходимо обратить внимание на процесс сбора данных.
По данным Пенсионного фонда, пенсионеры, выходящие на пенсию, имеют среднее значение трудового стажа 34,5 года, что соответствует 12592 дням, значения более 320000 дней многократно превышает эту цифру. Максимальный стаж указанный в таблице составляет 401755 если разделить это число на 24 и 365 полученный результат будет соответствовать 45 годам стажа, т.о. можно сделать вывод, что данные стажа свыше 320000 указаны в часах, приведем к единому виду:

In [1005]:
df['days_employed'] = df['days_employed'].apply(lambda x: x / 24 if x > 320000 else x)

Изменим знак стажа со значением меньше 0.

In [1006]:
df['days_employed'] = df['days_employed'].apply(lambda x: x*(-1) if x < 0 else x)

In [1007]:
df['days_employed'].describe()

count    21525.000000
mean      4172.840808
std       5267.376071
min          0.000000
25%        610.652074
50%       1808.053434
75%       4779.587738
max      18388.949901
Name: days_employed, dtype: float64

#### Оценка причины потери данных и обзор данных
Оценим, есть ли взаимосвязь пропущенных данных с другими столбцами, возможно пропущенные данные коррелируют с вводом какого-либо поля:

In [1008]:
#проверка на зависимость наличия пропусков от дней стажа
df[df['days_employed'] == 0]['income_type'].value_counts()

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64

In [1009]:
#проверка на зависимость наличия пропусков от образования
df[df['days_employed'] == 0]['education'].value_counts()

среднее                1408
высшее                  496
СРЕДНЕЕ                  67
Среднее                  65
неоконченное высшее      55
Высшее                   25
ВЫСШЕЕ                   23
начальное                19
Неоконченное высшее       7
НЕОКОНЧЕННОЕ ВЫСШЕЕ       7
НАЧАЛЬНОЕ                 1
Начальное                 1
Name: education, dtype: int64

In [1010]:
#проверка на зависимость наличия пропусков от непогашенной задолженности
df[df['days_employed'] == 0]['debt'].value_counts()

0    2004
1     170
Name: debt, dtype: int64

In [1011]:
#проверка на зависимость наличия пропусков от количества детей
df[df['days_employed'] == 0]['children'].value_counts()

 0     1439
 1      475
 2      204
 3       36
 20       9
 4        7
-1        3
 5        1
Name: children, dtype: int64

In [1012]:
#проверка на зависимость наличия пропусков от семейного положения
df[df['days_employed'] == 0]['family_status'].value_counts()

женат / замужем          1237
гражданский брак          442
Не женат / не замужем     288
в разводе                 112
вдовец / вдова             95
Name: family_status, dtype: int64

Согласно проверке методом info можно заметить, что пропуски симметричны в таблиице и их количество одинаково у обоих столбцов:

days_employed       2174

total_income        2174

Проверим соответствуют ли пропуски в столбцах друг другу: 

In [1013]:
#проверка на зависимость наличия пропусков в столбце days_employed и total_income
df[df['days_employed'] == 0]['total_income'].value_counts()

0.0    2174
Name: total_income, dtype: int64

Данные пропущенны в обоих столбцах соответственно.

#### Оценка влияния пропущенных значений на ключевые данные

Оценим соотношение количества пропущенных значений с информацией о просроченном кредите к общему количеству клиентов с задолженностью:

In [1014]:
print('{:.2%}'.format(df[df['total_income'] == 0]['debt'].sum() / df['debt'].sum()))

9.76%


В 9.76% строк с информацией о задолженности отсутствует информациия о доходах. Для исследования взаимосвязи уровня дохода и наличия задолженности эти данные могут повлиять на результат.
Заполним пропуски медианой зарплат предварительно сгруппировав клиентов по типу занятости.

In [1015]:
df_grouped = df.groupby('income_type').agg({'total_income':['mean'], 'days_employed':['mean']})
df_grouped

,total_income,days_employed
,mean,mean
income_type,,
безработный,131339.751676,15267.235531
в декрете,53829.130729,3296.759962
госслужащий,153679.631678,3057.343890
компаньон,182195.618704,1900.579581
пенсионер,122440.317524,13579.562374
предприниматель,249581.572474,260.424042
сотрудник,145342.380477,2095.293025
студент,98201.625314,578.751554


In [1016]:
group_income = df.groupby('income_type')['total_income'].median()

# заполним пропуски на медианный доход, основываясь на типе занятости
for i in group_income.index:
    df.loc[(df['income_type'] == i) & (df['total_income'].isnull()), 'total_income'] = group_income[i]


In [1017]:
group_days_employed = df.groupby('income_type')['days_employed'].median()

# заполним пропуски на медианный доход, основываясь на типе занятости
for i in group_days_employed.index:
    df.loc[(df['income_type'] == i) & (df['days_employed'].isnull()), 'days_employed'] = group_days_employed[i]


In [1018]:
#проверим наличие нулевых значений в столбце с доходом
df['total_income'].isna().sum()

0

In [1019]:
#проверим наличие нулевых значений в столбце со стажем
df['days_employed'].isna().sum()

0

Пропуски NaN отсутствуют и заменены на 0 в столбце days_employed и total_income. Для исследования взаимосвязи уровня дохода и задолженности, пропущенные значения заменены на медиану доходов. Также были приведены к общему виду значения столбца стажа.

Явной зависимости пропущенных данных от столбцов children, debt, education, income_type, family_status не обнаруженно.

 <b>Вывод:</b>

Данные отсутствуют у разных категорий клиентов, пропуски одинаково присутствуют в столбцах о стаже клиента и общем доходе, значимая зависимость с другими столбцами не выявлена, вероятнее всего присутствует случайная ошибка в записи данных. Необходимо обратить внимание разработчиков.

### Замена типа данных

Заменим вещественный тип данных на целочисленный методом для приведения типов данных библиотеки pandas - astype.

In [1020]:
df['days_employed'] = df['days_employed'].astype(int)
df['total_income'] = df['total_income'].astype(int)

Проверим результат:

In [1021]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


 <b>Вывод:</b>

В предоставленных данных значения в столбцах days_employed и total_income исправлены на целочисленные значения для удобства читаемости таблицы и снижения нагрузки на вычислительные мощности кода.

### Обработка дубликатов

Проверим наличие и количество дубликатов строк методом duplicated():

In [1022]:
df.duplicated().sum()

54

Обнаружен 54 дубликат. Так как в таблице встречались значения записанные с разным регистаром, прежде чем удалить дубликаты проверим все записи и приведем их к одной форме.

Проверим наличие дубликатов в столбце education методом value_counts(), для наглядного понимания количества значений в отличии от метода .unique():

In [1023]:
df['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

Приведем значения столбца education к одному регистру:

In [1024]:
df['education'] = df['education'].str.lower()
df['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

Проверим наличие дубликатов в столбце family_status:

In [1025]:
df['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

Также для наглядности приведем все значения к одному регистру:

In [1026]:
df['family_status'] = df['family_status'].str.lower()
df['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

Проверим наличие дубликатов в столбце gender:

In [1027]:
df['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

Обнаружен "артефакт", проверим информацию о клиенте с неверным значением в столбце пол:

In [1028]:
df[df['gender'] == 'XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,2358,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905,покупка недвижимости


Удалим строку из таблицы, значительно на результаты исследования это не повлияет.

In [1029]:
df = df.drop([10701])

Проверим результат:

In [1030]:
df['gender'].value_counts()

F    14236
M     7288
Name: gender, dtype: int64

Проверим наличие дубликатов в столбце income_type:

In [1031]:
df['income_type'].value_counts()

сотрудник          11119
компаньон           5084
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
студент                1
в декрете              1
Name: income_type, dtype: int64

Проверим наличие дубликатов в столбце purpose:

In [1032]:
df['purpose'].value_counts()

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
операции с жильем                         653
покупка жилья для сдачи                   653
операции с коммерческой недвижимостью     651
жилье                                     647
покупка жилья                             647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      623
покупка своего жилья                      620
строительство недвижимости                620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

Записи в столбце purpose внесены одним регистром, но представляют собой различные формулировки одной и той же цели. Приведение различающихся по форме, но одинаковых по смыслу значений к одной форме выполним ниже в разделе лемматизация.

Проверим количество дубликатов:

In [1033]:
df.duplicated().sum()

71

Найдены дополнительные дубликаты, удалим лишние строки.

In [1034]:
df = df.drop_duplicates().reset_index(drop = True)

Проверим результат:

In [1035]:
df.duplicated().sum()

0

 <b>Вывод:</b>

Дубликаты могли появиться вследствие сбоя в записи данных, либо повторного ввода данных пользователем, для более точного анализа причины не хватает данных. Стоит обратить внимание и разобраться с причинами появления такого «информационного мусора».

### Лемматизация

Выделим леммы из столбца purpose и проанализируем наиболее встречающиеся:

In [1036]:
#импортируем библиотеку pymystem3
from pymystem3 import Mystem
m = Mystem() 
purpose_list = df['purpose'].unique()
purpose_str = str(purpose_list)
df['purpose'].value_counts()

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
жилье                                     646
покупка жилья                             646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      620
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

In [1037]:
lemmas = m.lemmatize(purpose_str)
from collections import Counter
print(Counter(lemmas)) 

Counter({' ': 59, "' '": 23, "'\n": 14, " '": 14, 'покупка': 10, 'недвижимость': 10, 'автомобиль': 9, 'образование': 9, 'жилье': 7, 'с': 5, 'операция': 4, 'на': 4, 'свой': 4, 'свадьба': 3, 'строительство': 3, 'получение': 3, 'высокий': 3, 'дополнительный': 2, 'для': 2, 'коммерческий': 2, 'жилой': 2, 'подержать': 2, 'заниматься': 2, 'сделка': 2, "['": 1, 'приобретение': 1, 'сыграть': 1, 'проведение': 1, 'семья': 1, 'собственный': 1, 'со': 1, 'профильный': 1, 'сдача': 1, 'ремонт': 1, "']\n": 1})


В результате лемматизации можно выделить основные леммы: 'недвижимость', 'автомобиль', 'образование', встречающиеся чаще всего, а также более редкие: 'ремонт', 'строительство' и 'коммерческий'.

На основани выделенных лемм, заполним столбец с данными purpose следующими значениями:
- покупка коммерческой недвижимости,
- покупка недвижимости, 
- покупка автомобиля, 
- оплата образования,
- оплата свадьбы,
- ремонт жилища, 
- строительство дома.


In [1038]:
from nltk.stem import SnowballStemmer 
russian_stemmer = SnowballStemmer('russian') 
queries = purpose_list
stem_list = []
for query in queries:
    for word in query.split():
        stemmed_word = russian_stemmer.stem(word)
        stem_list.append(stemmed_word) 
print(Counter(stem_list))        

Counter({'покупк': 10, 'недвижим': 10, 'жил': 9, 'образован': 9, 'автомобил': 8, 'с': 5, 'операц': 4, 'на': 4, 'сво': 4, 'свадьб': 3, 'строительств': 3, 'получен': 3, 'высш': 3, 'дополнительн': 2, 'для': 2, 'коммерческ': 2, 'подержа': 2, 'заня': 2, 'сделк': 2, 'приобретен': 1, 'сыгра': 1, 'проведен': 1, 'сем': 1, 'собствен': 1, 'со': 1, 'автомоб': 1, 'профильн': 1, 'сдач': 1, 'ремонт': 1})


В результате получаем основы слов для анализа и приведения значений к стандартным:

In [1039]:
from nltk.stem import SnowballStemmer 
russian_stemmer = SnowballStemmer('russian')  

#функция приведения к стандартному виду в столбце purpose
def purpose_normalize(query):
    for word in query.split():
        stemmed_word = russian_stemmer.stem(word)
        if stemmed_word == 'ремонт':
            return 'ремонт жилища'        
        if stemmed_word == 'коммерческ':
            return 'покупка коммерческой недвижимости'
        if (stemmed_word == 'недвижим') or (stemmed_word == 'жил'):
            return 'покупка недвижимости'
        if (stemmed_word == 'автомобил') or(stemmed_word == 'автомоб'):
            return 'покупка автомобиля'
        if stemmed_word == 'свадьб':
            return 'оплата свадьбы'
        if stemmed_word == 'образован':
            return 'оплата образования'
        if stemmed_word == 'строительств':
            return 'строительство дома'
    return query
#применение функциии            
df['purpose'] = df['purpose'].apply(purpose_normalize)   

Проверим результат:

In [1040]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21453 entries, 0 to 21452
Data columns (total 12 columns):
children            21453 non-null int64
days_employed       21453 non-null int64
dob_years           21453 non-null int64
education           21453 non-null object
education_id        21453 non-null int64
family_status       21453 non-null object
family_status_id    21453 non-null int64
gender              21453 non-null object
income_type         21453 non-null object
debt                21453 non-null int64
total_income        21453 non-null int64
purpose             21453 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


Структура Data Frame не нарушена:

In [1041]:
print(df['purpose'].value_counts())

покупка недвижимости                 7014
покупка автомобиля                   4306
оплата образования                   4013
оплата свадьбы                       2324
строительство дома                   1878
покупка коммерческой недвижимости    1311
ремонт жилища                         607
Name: purpose, dtype: int64


### Проверка данных

После удаления дубликатов, проверим столбцы children, dob_years, total_income, days_employed на аномальные значения:
    

In [1042]:
df['children'].value_counts()

 0     14090
 1      4808
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Проверим есть ли клиенты младше 18 лет:

In [1043]:
df[df['dob_years'] < 18 ]['dob_years'].value_counts()

0    101
Name: dob_years, dtype: int64

Найдена 101 запись с пропущенным возрастом. Заменим на медиану в зависимости от статуса

In [1044]:
df_age_grouped = df.groupby('income_type').agg({'dob_years':['mean']})
df_age_grouped

,dob_years
,mean
income_type,
безработный,38.000000
в декрете,39.000000
госслужащий,40.647220
компаньон,39.696474
пенсионер,59.048577
предприниматель,42.500000
сотрудник,39.808372
студент,22.000000


In [1045]:
group_age = df.groupby('income_type')['dob_years'].median()
group_age
# заполним пропуски на медианный доход, основываясь на типе занятости
for i in group_age.index:
    df.loc[(df['income_type'] == i) & (df['dob_years']==0), 'dob_years'] = group_age[i]


Проверим результат:

In [1046]:
df[df['dob_years'] < 18 ]['dob_years'].value_counts()

Series([], Name: dob_years, dtype: int64)

Проверим максимальный возраст клиентов:

In [1047]:
df['dob_years'].max()

75.0

Максимальный возраст 75 лет.

Проверим нет ли отрицательных значений в столбце с информацией о доходах:

In [1048]:
df[df['total_income'] < 0 ]['total_income'].value_counts()

Series([], Name: total_income, dtype: int64)

Проверим нет ли отрицательных значений в столбце стажа:

In [1049]:
df[df['days_employed'] < 0 ]['days_employed'].value_counts()

Series([], Name: days_employed, dtype: int64)

На основе количества и разнообразия данных с отрицательным стажем работы и количеством детей можно сделать вывод, что отрицательное значение не единичная ошибка, вероятно при вводе данных знак использовался случайно.

Поменяем знак в столбцах days_employed и children:

In [1050]:
df['days_employed'] = abs(df['days_employed'])
df['children'] = abs(df['children'])

Проверим возраст клиентов с 20 детьми:

In [1051]:
df[df['children'] == 20]['dob_years'].value_counts()

56.0    5
40.0    4
37.0    4
50.0    3
45.0    3
34.0    3
42.0    3
49.0    3
30.0    3
46.0    3
31.0    2
35.0    2
43.0    2
39.0    2
59.0    2
27.0    2
33.0    2
32.0    2
36.0    2
29.0    2
44.0    2
41.0    2
23.0    1
64.0    1
26.0    1
24.0    1
52.0    1
69.0    1
54.0    1
55.0    1
60.0    1
53.0    1
57.0    1
61.0    1
48.0    1
25.0    1
38.0    1
62.0    1
51.0    1
21.0    1
Name: dob_years, dtype: int64

Предположим, что 0 дописан в результате ошибки. Заменим 20 на 2.

In [1052]:
df.loc[df['children'] == 20, 'children'] = 2

Выполним проверку:

In [1053]:
df[df['children'] == 20]['dob_years'].value_counts()

Series([], Name: dob_years, dtype: int64)

Аномально высокие и отрицательные значения заменены. 


 <b>Вывод:</b>

В результате лемматизации цели систематизированы для наглядного восприятия. Произведен анализ данных на аномальные значения, изменен знак значений со знаком минус. 

### Категоризация данных

#### По наличию задолженности
Создадим категории данных по наличию задолженности для удобства и читаемости таблицы при формировании сводной таблицы в следующем пункте:

In [1054]:
def is_debt(debt):
    if debt == 1:
        return 'кредит просрочен'
    if debt == 0:
        return 'кредит оплачен в срок'

df_debt = df
df_debt['is_debt'] = df_debt['debt'].apply(is_debt)

#### По уровню доходов
Для ответа на поставленные в исследовании вопросы выделим следующие категории клиентов:

- клиенты с высоким доходом
- клиенты со средним доходом
- клиенты с низким доходом

Разобьем клиентов на категории по уровню дохода:

In [1055]:
df['total_income'].describe().astype(int)

count      21453
mean      151008
std       109738
min            0
25%        89088
50%       135773
75%       195799
max      2265604
Name: total_income, dtype: int64

Средняя зарплата составляет 165262, медиана 145017.
Диапазон дохода от 20667 до 2265604.

In [1056]:
def total_income_cat(income):
    if income < 89000:
        return 'низкий'
    if 145000 >= income >= 135000:
        return 'ниже среднего'
    if 145000 <= income <= 151000:
        return 'выше среднего'
    if income > 195000:
        return 'высокий'
    

df_total_income = df
df_total_income['income_level'] = df_total_income['total_income'].apply(total_income_cat)
#сводная таблица
df_pivot_total_income = df.pivot_table(index=['income_level'], columns='is_debt', values='debt', aggfunc='count')
df_pivot_total_income


is_debt,кредит оплачен в срок,кредит просрочен
income_level,,
высокий,5033,389
выше среднего,522,54
ниже среднего,1027,109
низкий,4936,419


#### Категории клиенты с детьми
Добавим 2 категории для дальнейшего анализа и ответов на вопросы зависимости между наличием детей и оплатой кредита в срок
- клиенты с детьми
- клиенты без детей

In [1057]:
def is_children(children):
    if children > 0:
        return 'есть дети'
    if children == 0:
        return 'без детей'
    
df_children = df    
df_children['is_children'] = df_children['children'].apply(is_children)

 <b>Вывод:</b>

Выделены новые категории данных для удобства работы с таблицей и ответов на поставленные вопросы.

## Шаг 3. Ответьте на вопросы

### Есть ли зависимость между наличием детей и возвратом кредита в срок?

Для ответа на поставленный вопрос сгруппируем данные по количеству детей, рассчитаем соотношение количества просроченных кредитов к общему числу клиентов по каждой группе.

In [1058]:
#сгруппируем клиентов по количеству детей и посчитаем их общее количество
children_grouped_count = df.groupby('children')['debt'].count()

In [1059]:
#построим сводную таблицу:
children_pivot = df.pivot_table(index = ['children'], \
                                columns = ['debt'], \
                                values = 'purpose', aggfunc='count')
#рассчитаем отношениие клиентов с задолженностью ко всем остальным в категории
children_pivot['ratio'] = children_pivot[1] / children_grouped_count * 100
children_pivot = children_pivot.round(2).fillna(0)
children_pivot

debt,0,1,ratio
children,,,
0,13027.0,1063.0,7.54
1,4410.0,445.0,9.17
2,1926.0,202.0,9.49
3,303.0,27.0,8.18
4,37.0,4.0,9.76
5,9.0,0.0,0.00


По промежуточным результатам нельзя сказать, что клиенты с 5 детьми с меньшей вероятностью просрочат платежи, так как выборка не репрезентативна.

Проверим как обстоят дела в сумме:

In [1060]:
children_grouped_sum = df_children.groupby('is_children')['debt'].sum()
children_grouped_count = df_children.groupby('is_children')['debt'].count()
relation_children = children_grouped_sum / children_grouped_count * 100

relation_children.round(2)

is_children
без детей    7.54
есть дети    9.21
Name: debt, dtype: float64

 <b>Вывод:</b>

Как можно заметить присутствует зависимость между наличием детей и возвратом кредита в срок. Вероятность выплаты кредита в срок у клиентов без детей выше, чем у клиентов с детьми.

### Есть ли зависимость между семейным положением и возвратом кредита в срок?

Для ответа на поставленный вопрос сгруппируем данные по семейному положению, рассчитаем соотношение количества просроченных кредитов к общему числу клиентов по каждой группе.

In [1061]:
family_grouped_count = df.groupby('family_status')['debt'].count()

In [1062]:
family_status_pivot = df.pivot_table(index = ['family_status'], \
                                columns = ['debt'], \
                                values = 'purpose', aggfunc='count')
family_status_pivot['relation'] = family_status_pivot [1] / family_grouped_count * 100
family_status_pivot.round(2)

debt,0,1,relation
family_status,,,
в разводе,1110,85,7.11
вдовец / вдова,896,63,6.57
гражданский брак,3762,388,9.35
женат / замужем,11408,931,7.55
не женат / не замужем,2536,274,9.75


 <b>Вывод:</b>

Согласно полученным результатам наибольшая вероятность просрочить кредит у клиентов не состоящих в браке или состоящих в гражданском браке. Наименьшая вероятность у вдовы или вдовца, людей состоящих в браке и разведенных. Финансовое положение людей состоящих в законном браке более стабильно. Вероятно, у клиентов потерявших супруга на финансовое положение влияют страховые выплаты, пенсии.

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [1063]:
#сгруппируем клиентов по уровню дохода и посчитаем их общее количество
count_all = df_total_income.groupby('income_level')['debt'].count()
#рассчитаем количество клиентов с задолженностью в каждой из категории
sum_debt = df_total_income.groupby('income_level')['debt'].sum()
#рассчитаем отношениие клиентов с задолженностью ко всем остальным в категории
relation_income = sum_debt / count_all * 100
df_pivot_total_income['relation'] = relation_income.round(2)
df_pivot_total_income

is_debt,кредит оплачен в срок,кредит просрочен,relation
income_level,,,
высокий,5033,389,7.17
выше среднего,522,54,9.38
ниже среднего,1027,109,9.60
низкий,4936,419,7.82


 <b>Вывод:</b>

Согласно полученному результату вероятности задолженности у категорий клиентов с высоким и низким уровнем дохода отличаются незначительно. Клиенты с доходом ниже и выше среднего так же приблизительно равны. 

Вероятность не возврата кредита в срок у клиентов со средним доходом выше, чем у клиентов с высоким или низким уровнем дохода.

### Как разные цели кредита влияют на его возврат в срок?

Для ответа на поставленный вопрос сгруппируем данные по целям получения кредита, рассчитаем соотношение количества просроченных кредитов к общему числу клиентов по каждой группе.

In [1064]:
#сгруппируем клиентов по целям получениия кредита и посчитаем их общее количество
purposes_grouped_count = df.groupby('purpose')['debt'].count()
#рассчитаем отношениие клиентов с задолженностью ко всем остальным в группе
family_status_pivot = df.pivot_table(index = ['purpose'], \
                                columns = ['debt'], \
                                values = 'income_level', aggfunc='count')
family_status_pivot['relation'] = family_status_pivot[1] / purposes_grouped_count * 100
family_status_pivot.round(2)

debt,0,1,relation
purpose,,,
оплата образования,2140,191,4.76
оплата свадьбы,1214,106,4.56
покупка автомобиля,2304,230,5.34
покупка коммерческой недвижимости,728,67,5.11
покупка недвижимости,3783,276,3.93
ремонт жилища,340,18,2.97
строительство дома,1009,83,4.42


 <b>Вывод:</b>

Согласно полученным результатам получение кредита на оплату образования и покупку автомобиля имеют большую вероятность быть просроченным. Кредит на ремонт будет выплачен с большей вероятностью. 

## Шаг 4. Общий вывод

Вопросы стоящие перед исследованием данных:
- Есть ли зависимость между наличием детей и возвратом кредита в срок?
- Есть ли зависимость между семейным положением и возвратом кредита в срок?
- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
- Как разные цели кредита влияют на его возврат в срок?

Как показывает статистика присутствует зависимость между наличием детей и возвратом кредита в срок. Клиенты без детей реже имеют долги по кредитам. 

Семейное положение также влияет на количество кредитов погашенных в срок, вдовы, вдовцы, а также люди состоящие в браке и разведенные реже имеют просрочки по кредиту. Финансовое положение людей состоящих в законном браке более стабильно

Вероятность не возврата кредита в срок у клиентов со средним доходом выше, чем у клиентов с высоким или низким уровнем дохода.

Получение кредита на оплату образования и покупку автомобиля имеют большую вероятность быть не оплаченными в срок. Кредит на ремонт будет выплачен с большей вероятностью.